In [2]:
from sklearn import datasets, model_selection, svm, naive_bayes, tree, metrics
import pandas as pd

In [3]:
train = pd.read_csv('repairedTrain.csv')

iris = datasets.load_iris()
X = iris.data[:100, :2]
y = iris.target[:100]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1, random_state=0)

X = train.drop('class', axis=1)
y = train['class']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1, random_state=0)

# Support Vector Classifier

In [4]:
parameters = [{'kernel': ['rbf', 'poly'],
                   'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5],
                    'C': [1, 10, 100, 1000]},
              {'kernel': ['linear'], 
                   'C': [1, 10, 100, 1000]}]
clf = model_selection.GridSearchCV(svm.SVC(decision_function_shape='ovr'), parameters, cv=5, scoring = 'recall')

Nastavovanie parametrov pre cross-validaciu.<br>
Pri vyhodnocovani uspesnosti pri vsetkych klasifikatoroch sme si zvolili len metriku recall, pretoze nam ovela menej vadi povazovat zdraveho cloveka za choreho ako povazovat choreho za zdraveho. V nasom pripade minoritnou triedou su chori ludia a precision moze byt vysoky aj pri trivialnych klasifikatoroch.

In [5]:
clf.fit(X_train, y_train)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
#for mean, std, params in zip(means, stds, clf.cv_results_['params']):
#    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print(clf.best_params_)

ValueError: could not convert string to float: 'Own-child'

Najlepsie nastavenie hyperparametrov pre SVM klasifikator je C=1, gamma=0.1, kernel='rbf'.<br>

In [32]:
svc = svm.SVC(C=10, gamma=0.1, kernel='rbf') # kernel, C, gamma
svc.fit(X_train, y_train)

train_preds = svc.predict(X_train)
train_accuracy = metrics.accuracy_score(y_train, train_preds)
print("Vysledky na trenovacich datach\n", train_accuracy)
print(metrics.classification_report(y_train, train_preds), "\n\nVysledky na testovacich datach")

preds = svc.predict(X_test)
test_accuracy = metrics.accuracy_score(y_test, preds)
print(test_accuracy)
print(metrics.classification_report(y_test, preds))

Vysledky na trenovacich datach
 1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        47
          1       1.00      1.00      1.00        43

avg / total       1.00      1.00      1.00        90
 

Vysledky na testovacich datach
1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         3
          1       1.00      1.00      1.00         7

avg / total       1.00      1.00      1.00        10



<POPIS VYSLEDKOV...

In [33]:
#svc.support_vectors_ 

# Gaussian Naive Bayes

Pri naivnom bayesovskom klasifikatore niesu ziadne nastavilne hyperparametre ktore stoja za vyhladavanie gridsearchom

In [34]:
gnb = naive_bayes.GaussianNB().fit(X_train, y_train)

train_preds2 = gnb.predict(X_train)
train_accuracy = metrics.accuracy_score(y_train, train_preds2)
print(train_accuracy)
print(metrics.classification_report(y_train, train_preds2))

preds2 = gnb.predict(X_test)
test_accuracy = metrics.accuracy_score(y_test, preds2)
print(test_accuracy)
print(metrics.classification_report(y_test, preds2))

1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        47
          1       1.00      1.00      1.00        43

avg / total       1.00      1.00      1.00        90

1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         3
          1       1.00      1.00      1.00         7

avg / total       1.00      1.00      1.00        10



<POPIS VYSLEDKOV...

# Rozhodovaci strom

dtree = criterion (gini, entropy)<br>
max_depth = riesenie generalizacie<br>
min_samples_leaf = kolko pozorovani musi byt najmenej v tom liste (def=1 (malo), od 1 po 10 sa moze pouzivat cca

In [35]:
cv_params = {'max_depth': list(range(1,10)), 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 2, 4, 8]}
opt = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), cv_params, scoring='f1_macro', cv=5).fit(X_train, y_train)

In [36]:
means = opt.cv_results_['mean_test_score']
stds = opt.cv_results_['std_test_score']
#for mean, std, params in zip(means, stds, dtc.cv_results_['params']):
    #print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print(opt.best_params_)

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1}


Text o hyperparametroch...

In [37]:
dtc = tree.DecisionTreeClassifier(criterion='gini', max_depth=2, min_samples_leaf=1) # kernel, C, gamma
dtc.fit(X_train, y_train)

train_preds = dtc.predict(X_train)
train_accuracy = metrics.accuracy_score(y_train, train_preds)
print("Vysledky na trenovacich datach\n", train_accuracy)
print(metrics.classification_report(y_train, train_preds), "\n\nVysledky na testovacich datach")

preds = dtc.predict(X_test)
test_accuracy = metrics.accuracy_score(y_test, preds)
print(test_accuracy)
print(metrics.classification_report(y_test, preds))
dtc.feature_importances_  # cez toto vieme urcit ktore atributy boli nanic a sposobovali akysi sum... mozme ich zahodit

Vysledky na trenovacich datach
 0.9777777777777777
             precision    recall  f1-score   support

          0       0.98      0.98      0.98        47
          1       0.98      0.98      0.98        43

avg / total       0.98      0.98      0.98        90
 

Vysledky na testovacich datach
1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         3
          1       1.00      1.00      1.00         7

avg / total       1.00      1.00      1.00        10



array([0.65615486, 0.34384514])

Pre minoritne triedy by sme mohli vygenerovat navyse ich data pre zvysenie vahy, aby boli predikcie viac vybalancovane

1. train-test preprocess
2. Hyperparameter tuning
3. gridsearch -> najst optimalne parametre, vybranie metriky (recall)
4. porovnanie uspesnosti algs na zaklade noveho natrenovania a testovania na tren. a test. datach (na zistenie preucenia)
5. vyhodnotenie/porovnanie klasifikatorov, preco je ktory lepsi a kedy, ktore atributy sa pouzivali pri vyhodnocovani stromu

KOMENTARE ku kodom - preco robime, co to robi, preco to robi(ako dobre funguje), ake vysledky sa tam maju vidiet